# Open Demand

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## Import Database

In [ ]:
#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## Define clean_up

In [ ]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## Import Tables

In [ ]:
# Inventory
inventory_sql = """
SELECT 
    ItemCode
    , WhsCode
    , AVG(OnHand) AS Onhand
FROM 
	SAP.InventoryReportView
WHERE
	WhsCode = 'WW'
	AND OnHand > 0
GROUP BY
	ItemCode
	, WhsCode
"""

# Line Purchase Order
PO_sql = """
WITH PO AS
(SELECT 
	CASE
		WHEN Rate = 0 THEN 1
		ELSE Rate
	END AS EXCHANGE
	, ISBN
	, OpenQuantity
	, GrossPrice
	, ShippingCostUSD
	, Whse
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [DocumentStatus]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
FROM [SAP].[LinePurchaseOrderView]
)

SELECT
	[ISBN]
	, OpenQuantity
	, (GrossPrice*EXCHANGE +ShippingCostUSD) AS LandedCost
	, [Whse]
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
	, (CASE
		WHEN [RowDeliveryDate] > GETDATE() THEN 'OnTime'
		ELSE 'Late'
	END) AS DeliveryStatus
FROM 
	PO
WHERE 
	Whse in ('WW', 'TB') 
	AND OpenQuantity > 0
"""
# Line Sales Order

SO_sql = """
SELECT 
	Isbn
	, OpenQuantity
	, UnitPrice AS SalePrice
	, WarehouseCode
	, DeliveryDate
	, DocumentNumber
	, DocumentName
	, CustomerOrVendorCode
	, CustomerOrVendorName
	, Condition
	, (CASE
			WHEN DeliveryDate > GETDATE() THEN 'OnTime'
			ELSE 'Late'
		END) AS DeliveryStatus
FROM
	[SAP].[SaleOrderReportView]
WHERE OpenQuantity > 0 AND WarehouseCode IN ('WW')
"""

In [ ]:
inventory = pd.read_sql(inventory_sql, prod_db)
inventory.columns = map(str.lower, inventory.columns)
for col in ['itemcode', 'whscode']:
    inventory[col] = inventory.apply(lambda x: clean_up(x[col]), axis =1)
inventory.rename(columns={'itemcode': 'isbn'}, inplace = True)
print(inventory.isna().sum())
inventory.head()

In [ ]:
po = pd.read_sql(PO_sql, prod_db)
po.columns = map(str.lower, po.columns)
po.rename(columns={'bpname': 'supplier', 'bpcode': 'supplier_code', 'openquantity': 'po_open_qty', 'whse': 'po_whse', 'documentnumber' : 'po_number', 'documentname': ' po_name', 'deliverystatus': 'po_date_status'}, inplace = True)
for col in ['isbn', 'po_whse', ' po_name', 'supplier_code', 'supplier', 'orderstatus', 'po_date_status']:
    po[col] = po.apply(lambda x: clean_up(x[col]), axis =1)
print(po.isna().sum())
po.sort_values(['rowdeliverydate', 'po_whse', 'landedcost'], ascending=[True,False,True], inplace=True, ignore_index=True)
po.head()

In [ ]:
so = pd.read_sql(SO_sql, prod_db)
so.columns = map(str.lower, so.columns)
so.rename(columns={'openquantity': 'so_open_qty','warehousecode':'so_whse','customerorvendorname': 'customer', 'customerorvendorcode': 'customer_code', 'deliverydate' : 'customer_drop_date', 'documentnumber': 'so_number', 'documentname': 'so_name','deliverystatus': 'so_date_status'}, inplace = True)
for col in ['isbn', 'so_whse', 'so_name', 'customer', 'customer_code', 'condition', 'so_date_status']:
    so[col] = so.apply(lambda x: clean_up(x[col]), axis =1)
print(so.isna().sum())
so.sort_values(['customer_drop_date', 'saleprice'], ascending=[True,False], inplace=True, ignore_index=True)
so.head()

In [ ]:
so.columns

In [ ]:
#ALLOCATE RECEIVED INVENTORY INTO THE 

inv_so = pd.DataFrame()
for so_id in list(so[so['isbn'].isin(list(inventory['isbn']))].index.values):
    for inv_id in list(inventory[inventory['isbn'].isin(list(so['isbn']))].index.values):
        if (inventory.iloc[inv_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (inventory.iloc[inv_id, :]['onhand'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0):
            row_inv_so = pd.merge(inventory.iloc[[inv_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_inv_so['allocate_qty'] = row_inv_so[['onhand','so_open_qty']].min( axis = 1, skipna = True)
            inventory.loc[inv_id,'onhand'] = inventory.loc[inv_id,'onhand'] - row_inv_so.iloc[0,:]['allocate_qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_inv_so.iloc[0,:]['allocate_qty']
            row_inv_so['time_status'] = so.iloc[so_id, :]['so_date_status']
            row_inv_so['action'] = 'INVENTORY'
            inv_so = inv_so.append(row_inv_so)

In [ ]:
inv_so.head()

In [ ]:
po_so = pd.DataFrame()
for so_id in list(so[so['isbn'].isin(list(po['isbn'])) & (so['so_open_qty'] > 0)].index.values):
    for po_id in list(po[po['isbn'].isin(list(so['isbn'])) & (po['po_open_qty'] > 0)].index.values):
        if (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) >= 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH POSITIVE COMMISSION
            row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
            po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['allocate_qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['allocate_qty']
            row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
            # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
            # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE'
            # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE'
            # else:
            #     row_po_so_good['action'] = 'PROFIT'
        elif (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) < 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH NEGATIVE COMMISSION
            row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
            po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['allocate_qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['allocate_qty']
            row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
            # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
            # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE'
            # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
            #     row_po_so_good['action'] = 'UPDATE PO DATE'
            # else:
            #     row_po_so_good['action'] = 'NO PROFIT'
        po_so = po_so_good.append(row_po_so_good)

In [ ]:

po_so.drop(columns=['po_open_qty'. 'so_open_qty'], inplace=True)
po_so.head()

In [ ]:
# real_sale = pd.DataFrame()
# real_sale = inv_so.append(po_so_good)
po_so_good.to_csv('retail/po_so_good.csv', index= False)